In [1]:
import re
import urllib.request as req
import pandas

In [2]:
def extraction(content):
    name = []
    location = []
    date = []
    who = re.findall(r'itemprop="author">(.+?)</span>',content)
    for n in who:
        breakdown = re.match(r'(.+?) of (.+)',n)
        name.append(breakdown.group(1))
        location.append(breakdown.group(2))
        
    when = re.findall(r'<span class="review__post-date" content="(.+?)"',content)
    for t in when:
        breakdown = re.match(r'(\d{4}\-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})',t)
        date.append(pandas.to_datetime(breakdown.group(0),format='%Y-%m-%d %H:%M:%S'))
    what = re.findall(r'<div class="non-brand-campaign clearfix">(.*?)(<div class="">)*(<p>)*(.+?)</p>(<p>)*</div>',content)
    for x in range(len(what)):
        what[x] = what[x][3]
        what[x] = what[x].strip()
        what[x] = re.sub('<(.+?)>','',what[x])
    
    
    rating = re.findall(r'itemprop="ratingValue" content="([1-5])"',content)
    rating = rating = [int(x) for x in rating]
    rating = rating + [None for x in range(len(name) - len(rating))]
    
    data = {'name':name,'location':location,'datetime':date,'review':what,'rating':rating}
    result = pandas.DataFrame(data)
    return result

In [28]:
baseurl = 'https://www.consumeraffairs.com/finance/sallie_mae.html?page='
page = 1

resp = req.urlopen(baseurl + str(page))
webpage = resp.read()
webpage_fixed = webpage.decode('utf-8')
webpage_fixed = webpage_fixed.replace('\n','')
result = extraction(webpage_fixed)
page += 1

In [29]:
while(True):
    resp = req.urlopen(baseurl + str(page))
    actual_page = int(re.search(r'page=(\d+)',resp.url).groups(0)[0])
    if(page != actual_page):
        break
    webpage = resp.read()
    webpage_fixed = webpage.decode('utf-8')
    webpage_fixed = webpage_fixed.replace('\n','')
    extracted = extraction(webpage_fixed)
    result = pandas.concat([result,extracted],axis=0)
    print('Finished page ' + str(page) + ', Trying page ' + str(page+1))
    page += 1

Finished page 2, Trying page 3
Finished page 3, Trying page 4
Finished page 4, Trying page 5
Finished page 5, Trying page 6
Finished page 6, Trying page 7
Finished page 7, Trying page 8
Finished page 8, Trying page 9
Finished page 9, Trying page 10
Finished page 10, Trying page 11
Finished page 11, Trying page 12
Finished page 12, Trying page 13
Finished page 13, Trying page 14
Finished page 14, Trying page 15
Finished page 15, Trying page 16
Finished page 16, Trying page 17
Finished page 17, Trying page 18
Finished page 18, Trying page 19
Finished page 19, Trying page 20
Finished page 20, Trying page 21
Finished page 21, Trying page 22
Finished page 22, Trying page 23
Finished page 23, Trying page 24
Finished page 24, Trying page 25
Finished page 25, Trying page 26
Finished page 26, Trying page 27
Finished page 27, Trying page 28
Finished page 28, Trying page 29
Finished page 29, Trying page 30
Finished page 30, Trying page 31
Finished page 31, Trying page 32
Finished page 32, Trying 

In [31]:
result.reset_index(inplace=True,drop=True)
result.to_csv('consumer_affairs_sallie_mae.csv',index=False)
result.to_pickle('consumer_affairs_sallie_mae.data')